In [2]:
import sys  
sys.path.insert(0, '../..')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import optuna
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [5]:
experiment = []
run_id = 0

In [7]:
def objective(trial):
    seed = 0

    rock_results = []
    other_results = []

    n_samples = trial.suggest_int('n_samples', 1, 16) * 100
    jitter = trial.suggest_int('jitter', 1, 20) * 0.01
    factor = trial.suggest_int('factor', 1, 9) * 0.1

    datasets = []

    for s in range(seed, seed+10):
        datasets.append(make_circles(n_samples=n_samples,
                                     noise=jitter, 
                                     random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['run_id'] = run_id
        run['step'] = step
        run['n_samples'] = n_samples
        run['jitter'] = jitter
        run['factor'] = factor
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=2, random_state=seed).fit(data).labels_
        run['k'] = 2
        eps, min_pts = (0.2, 4)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts
        spectral = SpectralClustering(n_clusters=2, random_state=seed).fit(data).labels_
        bandwidth = estimate_bandwidth(data)
        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

    return np.mean(rock_results) - np.mean(other_results)

In [8]:
study_name = 'rings'

In [9]:
storage_name = f'sqlite:///../../results/optimization/{study_name}.db'
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name=study_name, storage=storage_name, sampler=sampler, load_if_exists=True,  direction="maximize")

[I 2021-05-30 14:47:56,346] A new study created in RDB with name: rings


In [10]:
study.optimize(objective, n_trials=100)

[I 2021-05-30 14:48:58,300] Trial 0 finished with value: -0.02142656584239766 and parameters: {'n_samples': 6, 'jitter': 20, 'factor': 7}. Best is trial 0 with value: -0.02142656584239766.
[I 2021-05-30 14:49:30,838] Trial 1 finished with value: 0.015381630723519255 and parameters: {'n_samples': 10, 'jitter': 4, 'factor': 2}. Best is trial 1 with value: 0.015381630723519255.
[I 2021-05-30 14:49:34,098] Trial 2 finished with value: -0.006095386668716002 and parameters: {'n_samples': 1, 'jitter': 18, 'factor': 6}. Best is trial 1 with value: 0.015381630723519255.
[I 2021-05-30 14:50:36,014] Trial 3 finished with value: -0.8377256182050731 and parameters: {'n_samples': 12, 'jitter': 1, 'factor': 9}. Best is trial 1 with value: 0.015381630723519255.
[I 2021-05-30 14:51:49,363] Trial 4 finished with value: 0.007022758619026565 and parameters: {'n_samples': 14, 'jitter': 5, 'factor': 2}. Best is trial 1 with value: 0.015381630723519255.
[I 2021-05-30 14:51:59,742] Trial 5 finished with value

[I 2021-05-30 15:52:53,427] Trial 41 finished with value: 0.17886297832745685 and parameters: {'n_samples': 15, 'jitter': 2, 'factor': 6}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 15:54:11,809] Trial 42 finished with value: -0.8413867837651066 and parameters: {'n_samples': 16, 'jitter': 1, 'factor': 9}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 15:55:17,904] Trial 43 finished with value: 0.1776092548696203 and parameters: {'n_samples': 14, 'jitter': 2, 'factor': 9}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 15:56:45,660] Trial 44 finished with value: 0.008401881671068551 and parameters: {'n_samples': 15, 'jitter': 6, 'factor': 6}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 15:56:51,209] Trial 45 finished with value: -0.05321677427280725 and parameters: {'n_samples': 1, 'jitter': 4, 'factor': 6}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 15:58:10,907] Trial 46 finished with value: 0.

[I 2021-05-30 16:43:48,578] Trial 82 finished with value: 0.06224135093179387 and parameters: {'n_samples': 16, 'jitter': 3, 'factor': 5}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 16:44:45,140] Trial 83 finished with value: -0.8413867837651066 and parameters: {'n_samples': 16, 'jitter': 1, 'factor': 4}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 16:46:03,888] Trial 84 finished with value: 0.06224135093179387 and parameters: {'n_samples': 16, 'jitter': 3, 'factor': 5}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 16:47:03,532] Trial 85 finished with value: 0.17886297832745685 and parameters: {'n_samples': 15, 'jitter': 2, 'factor': 4}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 16:48:49,352] Trial 86 finished with value: -0.83642090197778 and parameters: {'n_samples': 10, 'jitter': 1, 'factor': 5}. Best is trial 23 with value: 0.1789493922434452.
[I 2021-05-30 16:53:37,664] Trial 87 finished with value: 0.00

In [11]:
pd.DataFrame(experiment).to_csv('../../results/optimization/rings.csv')